In [50]:
#Import Libraries
import pandas as pd
import numpy as np

In [ ]:
#Loading Original Datasets

# Routes Dataset (All US)
original_all_routes = pd.read_json("./Original Datasets/openbeta-usa-routes-aug-2020.jsonlines", lines=True)

#Ratings Dataset
original_ratings = pd.read_pickle("./Original Datasets/All_Ratings.pkl.zip", compression= "zip") 

In [62]:
# Routes Dataset (All US)
edited_all_routes = original_all_routes

print(edited_all_routes.columns)

#Flatten grade into multiple columns
All_Grade_Categories = []
for index, row in edited_all_routes.iterrows():
    for grade_category in row['grade'].keys():
        if grade_category not in All_Grade_Categories:
            All_Grade_Categories.append(grade_category)

# print(All_Grade_Categories)


#Flatten type into multiple columns 
All_Type_Categories = []
for index, row in edited_all_routes.iterrows():
    for type_category in row['type'].keys():
        if type_category not in All_Type_Categories:
            All_Type_Categories.append(type_category)

# print(All_Type_Categories)

# Creating New Flattened Dataframe


route_list = []
for index, row in edited_all_routes.iterrows():
    #Create Base Dictionary
    route_dict = {}
    
    #Routename
    route_dict['route_name'] = row['route_name']

    #Flatten Grade
    for grade_key in All_Grade_Categories:
        if grade_key in row['grade'].keys():
            route_dict[grade_key] = row['grade'][grade_key]
        else:
            route_dict[grade_key] = np.NaN


    #Flatten type
    for type_key in All_Type_Categories:
        if type_key in row['type'].keys():
            route_dict[type_key] = row['type'][type_key]
        else:
            route_dict[type_key] = np.NaN

    #Route Id
    if len(row['metadata']['mp_route_id']) > 0:
        route_dict['route_id'] = row['metadata']['mp_route_id']
    else:
        continue

    #Location
    if len(row['location']) > 0:
        route_dict['location'] = row['location'][0]
    else:
        route_dict['location'] = np.NaN

    #fa
    route_dict['fa'] = row['fa']
    
    #Description
    if len(row['description']) > 0:
        route_dict['description'] = row['description'][0]
    else:
        route_dict['description'] = np.NaN

    #protection
    if len(row['protection']) > 0:
        route_dict['protection'] = row['protection'][0]
    else:
        route_dict['protection'] = np.NaN

    route_list.append(route_dict)


final_routes = pd.DataFrame(route_list).drop_duplicates(subset = ['route_id'])

final_routes.to_csv("./Initialize Table Datasets/routes.csv", index = False)


Index(['route_name', 'grade', 'safety', 'type', 'fa', 'description',
       'location', 'protection', 'metadata'],
      dtype='object')


In [95]:
#Ratings Dataset
edited_ratings = original_ratings

print(edited_ratings.shape[0])
rating_list = []

existing_routes = final_routes.route_id.values
print(existing_routes)


success_counter = 0

user_id_dict = {}
user_counter = 0

for index, row in edited_ratings.iterrows():
    rating_dict = {}

    #User_ID
    if row['users'] in user_id_dict.keys():
        rating_dict['user_id'] = user_id_dict[row['users']]
    else:
        user_counter += 1
        rating_dict['user_id'] = int(user_counter)
        user_id_dict[row['users']] = int(user_counter)
    
    #Route Id
    if f"{row['route_id']}" in existing_routes:
        success_counter += 1
        print(f"Success Counter: {success_counter} Route_ID: {row['route_id']}")
        rating_dict['route_id'] = row['route_id']
    else:
        continue

    #Rating
    rating_dict['rating'] = row['ratings']

    rating_list.append(rating_dict)

    #Rating_ID 
    rating_dict['rating_id'] = success_counter


final_ratings = pd.DataFrame(rating_list)

print(final_ratings)
final_ratings.to_csv("./Initialize Table Datasets/ratings.csv", index = False)



10000
['118170758' '107415908' '117944541' ... '107181585' '111824794'
 '108558242']
Success Counter: 1 Route_ID: 106528796
Success Counter: 2 Route_ID: 106528796
Success Counter: 3 Route_ID: 115405532
Success Counter: 4 Route_ID: 117653356
Success Counter: 5 Route_ID: 117653356
Success Counter: 6 Route_ID: 107821705
Success Counter: 7 Route_ID: 107821705
Success Counter: 8 Route_ID: 107821705
Success Counter: 9 Route_ID: 107821689
Success Counter: 10 Route_ID: 107821689
Success Counter: 11 Route_ID: 107821689
Success Counter: 12 Route_ID: 107821724
Success Counter: 13 Route_ID: 107821724
Success Counter: 14 Route_ID: 107821724
Success Counter: 15 Route_ID: 107836258
Success Counter: 16 Route_ID: 107836258
Success Counter: 17 Route_ID: 107836258
Success Counter: 18 Route_ID: 107821666
Success Counter: 19 Route_ID: 107821666
Success Counter: 20 Route_ID: 107821747
Success Counter: 21 Route_ID: 107821747
Success Counter: 22 Route_ID: 107821747
Success Counter: 23 Route_ID: 107821747
Succ

In [94]:
#User Table

user_list = []

for index, row in final_ratings.iterrows():
    user_dict = {}

    user_dict['user_id'] = int(row['user_id'])

    user_dict['name'] = np.NaN
    user_dict['email'] = np.NaN
    user_dict['age'] = np.NaN

    user_list.append(user_dict)

final_users = pd.DataFrame(user_list).drop_duplicates(subset = ['user_id'])

print(final_ratings)
final_users.to_csv("./Initialize Table Datasets/users.csv", index = False)

      user_id   route_id  rating  rating_id
0           1  106528796     3.0          1
1           2  106528796     3.0          2
2           3  115405532     3.0          3
3           4  117653356     3.0          4
4           5  117653356     2.0          5
...       ...        ...     ...        ...
9995     1173  105878191     2.0       9996
9996      902  105878191     2.0       9997
9997     1202  105878191     2.0       9998
9998       46  105878191     2.0       9999
9999      105  105878191     2.0      10000

[10000 rows x 4 columns]
